In [1]:
!git clone https://github.com/hakunamatata1997/Image-Enhancer-WebApp.git

Cloning into 'Image-Enhancer-WebApp'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 53 (delta 13), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (53/53), 1.62 MiB | 17.42 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [4]:
%ls

app.py  examples/  LICENSE  README.md  requirements.txt  static/  templates/


In [3]:
%cd Image-Enhancer-WebApp/

/content/Image-Enhancer-WebApp


In [10]:
!pip install -r requirements.txt

In [11]:
!pip install -U BasicSR

In [6]:
!pip show torchvision

Name: torchvision
Version: 0.17.1+cu121
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, torch
Required-by: basicsr, facexlib, fastai, gfpgan, realesrgan


In [7]:
!pip install flask-ngrok flask pyngrok
!ngrok authtoken '2c5FrWWOH0G5WlkyjcCG2OOdJjW_5TQd7A9yg5PYD3ksSpVdG' # 填自己Authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
import requests
import sys
import shutil
def fix():
  url = "https://raw.githubusercontent.com/neuralfalcon/DreamTalk-Colab-Demo/main/degradations.py"
  filename = "/content/degradations.py"
  full_version = sys.version.split(' ')[0]
  major_minor_version = '.'.join(full_version.split('.')[:2])
  basicsr_path=f"/usr/local/lib/python{major_minor_version}/dist-packages/basicsr/data/degradations.py"
  # Send a GET request to the URL
  response = requests.get(url)
  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Write the content to a file
      with open(filename, 'wb') as file:
          file.write(response.content)
      try:
        shutil.copy("/content/degradations.py",basicsr_path)
        print(f"Copied to {basicsr_path}")
      except:
        print("Update the 'basicsr_path' variable -- replace '{major_minor_version}' with the current Python version in Google Colab, such as 3.10.")

  else:
      print("Failed to download the file.")
fix()

Copied to /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py


In [14]:
import os
from flask import Flask, render_template, request
import cv2
import torch
from basicsr.archs.srvgg_arch import SRVGGNetCompact
from gfpgan.utils import GFPGANer
from realesrgan.utils import RealESRGANer

from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)
# Set the upload folder and allowed extensions
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['ALLOWED_EXTENSIONS'] = {'jpg', 'jpeg', 'png'}

# Create the uploads and ouputs folder if it does not exist
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs('static/outputs', exist_ok=True)

# Download weights if not available
model_weights = {
    'realesr-general-x4v3.pth': 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth',
    'GFPGANv1.2.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.2.pth',
    'GFPGANv1.3.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
    'GFPGANv1.4.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth',
    'RestoreFormer.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/RestoreFormer.pth',
    'CodeFormer.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/CodeFormer.pth'
}

for weight_file, weight_url in model_weights.items():
    if not os.path.exists(weight_file):
        os.system(f"wget {weight_url} -P .")

# Create the background enhancer with RealESRGAN
model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=32, upscale=4, act_type='prelu')
model_path = 'realesr-general-x4v3.pth'
half = True if torch.cuda.is_available() else False
upsampler = RealESRGANer(scale=4, model_path=model_path, model=model, tile=0, tile_pad=10, pre_pad=0, half=half)



def allowed_file(filename):
    """Check if the file extension is allowed."""
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']


def enhance_image(image_path, version, scale):
    filename = os.path.splitext(os.path.basename(image_path))[0]
    try:
        if scale > 4:
            scale = 4  # avoid too large scale value

        extension = os.path.splitext(os.path.basename(image_path))[1]
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if len(img.shape) == 3 and img.shape[2] == 4:
            img_mode = 'RGBA'
        elif len(img.shape) == 2:  # for gray inputs
            img_mode = None
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        else:
            img_mode = None

        h, w = img.shape[0:2]
        if h > 3500 or w > 3500:
            print('Too large size')
            return None, None

        if h < 300:
            img = cv2.resize(img, (w * 2, h * 2), interpolation=cv2.INTER_LANCZOS4)

        if version == 'v1.2':
            face_enhancer = GFPGANer(
                model_path='GFPGANv1.2.pth', upscale=2, arch='clean', channel_multiplier=2, bg_upsampler=upsampler)
        elif version == 'v1.3':
            face_enhancer = GFPGANer(
                model_path='GFPGANv1.3.pth', upscale=2, arch='clean', channel_multiplier=2, bg_upsampler=upsampler)
        elif version == 'v1.4':
            face_enhancer = GFPGANer(
                model_path='GFPGANv1.4.pth', upscale=2, arch='clean', channel_multiplier=2, bg_upsampler=upsampler)
        elif version == 'RestoreFormer':
            face_enhancer = GFPGANer(
                model_path='RestoreFormer.pth', upscale=2, arch='RestoreFormer', channel_multiplier=2, bg_upsampler=upsampler)
        elif version == 'CodeFormer':
            face_enhancer = GFPGANer(
                model_path='CodeFormer.pth', upscale=2, arch='CodeFormer', channel_multiplier=2, bg_upsampler=upsampler)

        try:
            _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
        except RuntimeError as error:
            print('Error:', error)

        try:
            if scale != 2:
                interpolation = cv2.INTER_AREA if scale < 2 else cv2.INTER_LANCZOS4
                h, w = img.shape[0:2]
                output = cv2.resize(output, (int(w * scale / 2), int(h * scale / 2)), interpolation=interpolation)
        except Exception as error:
            print('Wrong scale input.', error)

        if img_mode == 'RGBA':  # RGBA images should be saved in png format
            extension = 'png'
        else:
            extension = 'jpg'
        output_path = f'static/outputs/{filename}.{extension}'
        cv2.imwrite(output_path, output)

        return output_path, image_path

    except Exception as error:
        print('Global exception:', error)
        return None, None



@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Check if the POST request has the file part
        if 'file' not in request.files:
            return render_template('index.html', error='No file part')

        file = request.files['file']
        version = request.form['version']
        scale = int(request.form['scale'])

        # Validate file
        if file.filename == '':
            return render_template('index.html', error='No file selected')
        if not allowed_file(file.filename):
            return render_template('index.html', error='Invalid file type')

        # Save the file
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)

        # Perform image enhancement
        output_path, input_path = enhance_image(file_path, version, scale)

        # Render the result page with image paths
        return render_template('result.html', before_image_path=input_path, after_image_path=output_path)

    return render_template('index.html')


if __name__ == '__main__':

    app.run()





 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [ ]:
import os
from flask import Flask, render_template, request
import cv2
import torch
from basicsr.archs.srvgg_arch import SRVGGNetCompact
from gfpgan.utils import GFPGANer
from realesrgan.utils import RealESRGANer

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))


from flask import Flask, render_template
app = Flask(__name__)
# Set the upload folder and allowed extensions
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['ALLOWED_EXTENSIONS'] = {'jpg', 'jpeg', 'png'}

# Create the uploads and ouputs folder if it does not exist
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs('static/outputs', exist_ok=True)

# Download weights if not available
model_weights = {
    'realesr-general-x4v3.pth': 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth',
    'GFPGANv1.2.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.2.pth',
    'GFPGANv1.3.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
    'GFPGANv1.4.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth',
    'RestoreFormer.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/RestoreFormer.pth',
    'CodeFormer.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/CodeFormer.pth'
}

for weight_file, weight_url in model_weights.items():
    if not os.path.exists(weight_file):
        os.system(f"wget {weight_url} -P .")

# Create the background enhancer with RealESRGAN
model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=32, upscale=4, act_type='prelu')
model_path = 'realesr-general-x4v3.pth'
half = True if torch.cuda.is_available() else False
upsampler = RealESRGANer(scale=4, model_path=model_path, model=model, tile=0, tile_pad=10, pre_pad=0, half=half)



def allowed_file(filename):
    """Check if the file extension is allowed."""
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']


def enhance_image(image_path, version, scale):
    filename = os.path.splitext(os.path.basename(image_path))[0]
    try:
        if scale > 4:
            scale = 4  # avoid too large scale value

        extension = os.path.splitext(os.path.basename(image_path))[1]
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if len(img.shape) == 3 and img.shape[2] == 4:
            img_mode = 'RGBA'
        elif len(img.shape) == 2:  # for gray inputs
            img_mode = None
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        else:
            img_mode = None

        h, w = img.shape[0:2]
        if h > 3500 or w > 3500:
            print('Too large size')
            return None, None

        if h < 300:
            img = cv2.resize(img, (w * 2, h * 2), interpolation=cv2.INTER_LANCZOS4)

        if version == 'v1.2':
            face_enhancer = GFPGANer(
                model_path='GFPGANv1.2.pth', upscale=2, arch='clean', channel_multiplier=2, bg_upsampler=upsampler)
        elif version == 'v1.3':
            face_enhancer = GFPGANer(
                model_path='GFPGANv1.3.pth', upscale=2, arch='clean', channel_multiplier=2, bg_upsampler=upsampler)
        elif version == 'v1.4':
            face_enhancer = GFPGANer(
                model_path='GFPGANv1.4.pth', upscale=2, arch='clean', channel_multiplier=2, bg_upsampler=upsampler)
        elif version == 'RestoreFormer':
            face_enhancer = GFPGANer(
                model_path='RestoreFormer.pth', upscale=2, arch='RestoreFormer', channel_multiplier=2, bg_upsampler=upsampler)
        elif version == 'CodeFormer':
            face_enhancer = GFPGANer(
                model_path='CodeFormer.pth', upscale=2, arch='CodeFormer', channel_multiplier=2, bg_upsampler=upsampler)

        try:
            _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
        except RuntimeError as error:
            print('Error:', error)

        try:
            if scale != 2:
                interpolation = cv2.INTER_AREA if scale < 2 else cv2.INTER_LANCZOS4
                h, w = img.shape[0:2]
                output = cv2.resize(output, (int(w * scale / 2), int(h * scale / 2)), interpolation=interpolation)
        except Exception as error:
            print('Wrong scale input.', error)

        if img_mode == 'RGBA':  # RGBA images should be saved in png format
            extension = 'png'
        else:
            extension = 'jpg'
        output_path = f'static/outputs/{filename}.{extension}'
        cv2.imwrite(output_path, output)

        return output_path, image_path

    except Exception as error:
        print('Global exception:', error)
        return None, None



@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Check if the POST request has the file part
        if 'file' not in request.files:
            return render_template('index.html', error='No file part')

        file = request.files['file']
        version = request.form['version']
        scale = int(request.form['scale'])

        # Validate file
        if file.filename == '':
            return render_template('index.html', error='No file selected')
        if not allowed_file(file.filename):
            return render_template('index.html', error='Invalid file type')

        # Save the file
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)

        # Perform image enhancement
        output_path, input_path = enhance_image(file_path, version, scale)

        # Render the result page with image paths
        return render_template('result.html', before_image_path=input_path, after_image_path=output_path)

    return render_template('index.html')
if __name__ == '__main__':
    app.run()

https://yjzltkj45l-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:31:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:31:53] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:31:54] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/Image-Enhancer-WebApp/gfpgan/weights/detection_Resnet50_Final.pth



100%|██████████| 104M/104M [00:00<00:00, 149MB/s]


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /content/Image-Enhancer-WebApp/gfpgan/weights/parsing_parsenet.pth



100%|██████████| 81.4M/81.4M [00:00<00:00, 180MB/s]
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:28] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:28] "GET /static/outputs/loopy表情包大分享！_1_123_来自小红书网页版.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:28] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:28] "GET /static/uploads/loopy表情包大分享！_1_123_来自小红书网页版.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:29] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:38] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:47] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 04:32:47] "GET /static/outputs/loopy表情包大分享！_1_123_来自小红书网页版.jpg HTTP/1.1" 200 -
INFO:werkzeug: